In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import math

In [19]:
discussions_df = pd.read_csv('data/Discussions.csv', header=0, encoding='utf-8')
ratings_df = pd.read_csv('data/Ratings.csv', header=0, encoding='utf-8')
display(discussions_df)
display(ratings_df)

,DiscussionID,AlbumName,ArtistName,Date,AvgRating,Stdev,Attendance,RotationGenre,OtherGenre,Subgenres,ReleaseYear,FavoriteTrack,Popularity,Tracks,SpotifyID,Image
0,1,IGOR,"Tyler, the Creator",2/19/2022,0.00,0.00,0,NaN,False,"Neo-Soul, Psychedelic Soul",2019,NaN,85,12,5zi7WsKlIiUXv09tbGLKsE,https://i.scdn.co/image/ab67616d0000b273700588...
1,2,The New Abnormal,The Strokes,2/26/2022,0.00,0.00,0,NaN,False,Indie Rock,2020,NaN,76,9,2xkZV2Hl1Omi8rk2D7t5lN,https://i.scdn.co/image/ab67616d0000b273bfa99a...
2,3,Blonde,Frank Ocean,3/6/2022,0.00,0.00,0,NaN,False,"Alternative R&B, Art Pop, Neo-Soul",2016,NaN,86,17,3mH6qwIy9crq0I9YQbOuDf,https://i.scdn.co/image/ab67616d0000b273c5649a...
3,4,So Tonight that I Might See,Mazzy Star,3/27/2022,0.00,0.00,0,NaN,False,"Dream Pop, Neo-Psychedelia",1993,NaN,73,10,5K18gTgac0q6Jma5HkV1vA,https://i.scdn.co/image/ab67616d0000b27389a392...
4,5,A New World Record,Electric Light Orchestra,4/3/2022,0.00,0.00,0,NaN,False,"Symphonic Rock, Pop Rock, Progressive Pop",1976,NaN,67,9,7a35UzxXYuKQGMGImyB0Un,https://i.scdn.co/image/ab67616d0000b273ee5b10...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,66,Turn On The Bright Lights,Interpol,1/29/2024,7.40,1.60,57,Rock,False,"Post-Punk Revival, Indie Rock",2002,Obstacle 1,66,11,4sW8Eql2e2kdRP1A1R1clG,https://i.scdn.co/image/ab67616d00001e0214f8df...
66,67,Titanic Rising,Weyes Blood,2/5/2024,8.20,1.50,55,Pop,False,"Baroque Pop, Art Pop, Progressive Pop, Singer-...",2019,Andromeda,68,10,0Cuqhgy8vm96JEkBY3polk,https://i.scdn.co/image/ab67616d00001e020c64e7...
67,68,Because the Internet,Childish Gambino,2/12/2024,6.88,1.93,42,Hip-Hop,False,"Abstract Hip Hop, West Coast Hip Hop, Contempo...",2013,3005,81,19,62yjWIhnATHxPqGbgC9Lqr,https://i.scdn.co/image/ab67616d00001e0226d64b...
68,69,The Four Seasons,Antonio Vivaldi,2/19/2024,8.08,1.25,38,Classical,True,"Baroque Music, Concerto",1985,Winter: I. Allegro non molto,63,12,4eEJooUkzenp7lZTtfbIP7,https://i.scdn.co/image/ab67616d00001e02c7464f...


,MemberID,DiscussionID,Rating,FavoriteTrack1,FavoriteTrack2,FavoriteTrack3
0,88,1,7,NaN,NaN,NaN
1,59,2,8,NaN,NaN,NaN
2,88,3,9,NaN,NaN,NaN
3,294,3,10,NaN,NaN,NaN
4,201,3,7,NaN,NaN,NaN
...,...,...,...,...,...,...
1707,313,69,8,"Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ..."
1708,165,69,7,NaN,NaN,NaN
1709,131,69,8,"Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ..."
1710,286,69,7,"Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ..."


# Idea 1: Subgenres

In the AOTW database, we already have a column "Genre" which we use to designate different rotation cycles (rock, pop, hip hop, other/special, wildcard) so that we see a wide variety of genres in the club. We store a specific album's genres/styles as "subgenres" in the AOTW database, with genres provided by RateYourMusic.

In [22]:
subgenres_series = discussions_df['Subgenres'].str.split(', ').explode()

# Get unique subgenres and sort them alphabetically
subgenres_series = subgenres_series.dropna()
unique_subgenres = sorted(subgenres_series.unique())

# Print unique subgenres
print("Unique Subgenres (Alphabetically Sorted):")
for subgenre in unique_subgenres:
    print(subgenre)

Unique Subgenres (Alphabetically Sorted):
Abstract Hip Hop
Alt-Pop
Alternative Dance
Alternative R&B
Alternative Rock
Ambient
Art Pop
Art Rock
Baroque Music
Baroque Pop
Britpop
Bubblegum Bass
Cambodian Pop
Chamber Pop
Concerto
Conscious Hip Hop
Contemporary Folk
Contemporary R&B
Country Rock
Dance-Pop
Dance-Punk
Deconstructed Club
Downtempo
Dream Pop
Drug-Spiral Rap
East Coast Hip Hop
Electric Blues
Electronic
Electropop
Emo
Experimental
Experimental Hip Hop
Experimental Rock
Folk Rock
Glam Rock
Glitch Hop
Gothic Rock
Hard Rock
Hardcore Hip Hop
Indie Folk
Indie Pop
Indie Rock
Indietronica
Instrumental Hip Hop
Jazz Fusion
Jazz Rap
Jazz-Rock
Midwest Emo
Neo-Psychedelia
Neo-Soul
New Wave
Plunderphonics
Pop
Pop Rock
Pop Soul
Post-Punk
Post-Punk Revival
Post-Rock
Power Pop
Progressive Pop
Progressive Rock
Psychedelic Pop
Psychedelic Soul
Rage
Rhythm & Blues
Shoegaze
Singer-Songwriter
Soft Rock
Soul
Soul Blues
Southern Hip Hop
Standards
Symphonic Rock
Synthpop
Trap
Trip Hop
UK Hip Hop
Video 

In [40]:
merged_df = pd.merge(discussions_df, ratings_df, on='DiscussionID')

# Split the Subgenres column and explode it into multiple rows
merged_df['Subgenre'] = merged_df['Subgenres'].str.split(', ')
merged_df = merged_df.explode('Subgenre')
display(merged_df)

# Group by MemberID and Subgenre, then calculate average rating
average_ratings_by_subgenre = merged_df.groupby(['MemberID', 'Subgenre'])['Rating'].mean().reset_index()

# Display the DataFrame with average ratings by subgenre
for idx, row in average_ratings_by_subgenre.iterrows():
    print(f"{row['MemberID']}\t{row['Subgenre']}\t{row['Rating']}")

,DiscussionID,AlbumName,ArtistName,Date,AvgRating,Stdev,Attendance,RotationGenre,OtherGenre,Subgenres,...,Popularity,Tracks,SpotifyID,Image,MemberID,Rating,FavoriteTrack1,FavoriteTrack2,FavoriteTrack3,Subgenre
0,1,IGOR,"Tyler, the Creator",2/19/2022,0.00,0.00,0,NaN,False,"Neo-Soul, Psychedelic Soul",...,85,12,5zi7WsKlIiUXv09tbGLKsE,https://i.scdn.co/image/ab67616d0000b273700588...,88,7,NaN,NaN,NaN,Neo-Soul
0,1,IGOR,"Tyler, the Creator",2/19/2022,0.00,0.00,0,NaN,False,"Neo-Soul, Psychedelic Soul",...,85,12,5zi7WsKlIiUXv09tbGLKsE,https://i.scdn.co/image/ab67616d0000b273700588...,88,7,NaN,NaN,NaN,Psychedelic Soul
1,2,The New Abnormal,The Strokes,2/26/2022,0.00,0.00,0,NaN,False,Indie Rock,...,76,9,2xkZV2Hl1Omi8rk2D7t5lN,https://i.scdn.co/image/ab67616d0000b273bfa99a...,59,8,NaN,NaN,NaN,Indie Rock
2,3,Blonde,Frank Ocean,3/6/2022,0.00,0.00,0,NaN,False,"Alternative R&B, Art Pop, Neo-Soul",...,86,17,3mH6qwIy9crq0I9YQbOuDf,https://i.scdn.co/image/ab67616d0000b273c5649a...,88,9,NaN,NaN,NaN,Alternative R&B
2,3,Blonde,Frank Ocean,3/6/2022,0.00,0.00,0,NaN,False,"Alternative R&B, Art Pop, Neo-Soul",...,86,17,3mH6qwIy9crq0I9YQbOuDf,https://i.scdn.co/image/ab67616d0000b273c5649a...,88,9,NaN,NaN,NaN,Art Pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709,69,The Four Seasons,Antonio Vivaldi,2/19/2024,8.08,1.25,38,Classical,True,"Baroque Music, Concerto",...,63,12,4eEJooUkzenp7lZTtfbIP7,https://i.scdn.co/image/ab67616d00001e02c7464f...,131,8,"Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...",Concerto
1710,69,The Four Seasons,Antonio Vivaldi,2/19/2024,8.08,1.25,38,Classical,True,"Baroque Music, Concerto",...,63,12,4eEJooUkzenp7lZTtfbIP7,https://i.scdn.co/image/ab67616d00001e02c7464f...,286,7,"Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...",Baroque Music
1710,69,The Four Seasons,Antonio Vivaldi,2/19/2024,8.08,1.25,38,Classical,True,"Baroque Music, Concerto",...,63,12,4eEJooUkzenp7lZTtfbIP7,https://i.scdn.co/image/ab67616d00001e02c7464f...,286,7,"Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...",Concerto
1711,69,The Four Seasons,Antonio Vivaldi,2/19/2024,8.08,1.25,38,Classical,True,"Baroque Music, Concerto",...,63,12,4eEJooUkzenp7lZTtfbIP7,https://i.scdn.co/image/ab67616d00001e02c7464f...,19,7,"Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...",Baroque Music


1	Abstract Hip Hop	7.0
1	Alternative R&B	9.5
1	Alternative Rock	9.0
1	Ambient	7.0
1	Art Pop	8.5
1	Art Rock	8.666666666666666
1	Baroque Pop	9.0
1	Britpop	9.0
1	Bubblegum Bass	7.0
1	Cambodian Pop	8.0
1	Chamber Pop	8.0
1	Conscious Hip Hop	9.333333333333334
1	Contemporary Folk	8.0
1	Contemporary R&B	8.0
1	Country Rock	7.0
1	Deconstructed Club	7.0
1	Dream Pop	7.5
1	Drug-Spiral Rap	7.0
1	East Coast Hip Hop	8.0
1	Electric Blues	7.0
1	Electronic	9.0
1	Electropop	5.0
1	Emo	7.0
1	Experimental Hip Hop	8.6
1	Experimental Rock	10.0
1	Folk Rock	7.0
1	Glam Rock	9.0
1	Gothic Rock	8.0
1	Hardcore Hip Hop	7.0
1	Indie Pop	10.0
1	Indie Rock	8.25
1	Indietronica	10.0
1	Instrumental Hip Hop	9.0
1	Jazz Fusion	7.0
1	Jazz Rap	8.0
1	Jazz-Rock	7.0
1	Midwest Emo	10.0
1	Neo-Psychedelia	9.0
1	Neo-Soul	9.0
1	New Wave	9.0
1	Plunderphonics	9.0
1	Pop	7.0
1	Pop Rock	8.5
1	Pop Soul	10.0
1	Post-Punk	9.0
1	Post-Punk Revival	8.0
1	Post-Rock	9.0
1	Power Pop	10.0
1	Progressive Pop	8.0
1	Progressive Rock	8.5
1	Psychedelic Pop	8.

52	Progressive Rock	7.0
52	Psychedelic Pop	8.0
52	Rage	4.0
52	Rhythm & Blues	9.0
52	Singer-Songwriter	9.0
52	Soft Rock	9.0
52	Soul	9.0
52	Southern Hip Hop	6.0
52	Symphonic Rock	7.0
52	Synthpop	8.0
52	Trap	4.0
52	Trip Hop	6.0
52	Video Game Music	6.0
52	West Coast Hip Hop	8.0
53	Contemporary R&B	10.0
53	East Coast Hip Hop	10.0
53	Neo-Soul	10.0
53	Pop Rock	9.0
53	Soft Rock	9.0
54	Emo	9.0
54	Indie Rock	9.0
54	Pop Rock	5.0
54	Shoegaze	9.0
54	Soft Rock	5.0
55	Alternative R&B	6.0
55	Contemporary R&B	6.0
55	Pop Rock	8.0
55	Soft Rock	8.0
56	Pop Rock	8.0
56	Soft Rock	8.0
57	Alternative R&B	10.0
57	Contemporary R&B	10.0
57	Pop Rock	5.0
57	Soft Rock	5.0
58	Alternative R&B	8.0
58	Contemporary R&B	8.0
58	Pop Rock	6.0
58	Soft Rock	6.0
59	Abstract Hip Hop	6.0
59	Alt-Pop	7.0
59	Alternative Dance	9.0
59	Alternative R&B	8.0
59	Alternative Rock	9.666666666666666
59	Ambient	10.0
59	Art Pop	8.333333333333334
59	Art Rock	8.0
59	Baroque Music	9.0
59	Baroque Pop	10.0
59	Britpop	7.0
59	Bubblegum Bass	7.0
59	Cam

145	Britpop	5.0
145	Bubblegum Bass	4.0
145	Conscious Hip Hop	7.0
145	Deconstructed Club	4.0
145	Dream Pop	8.0
145	Electropop	8.0
145	Indie Pop	9.0
145	Indie Rock	8.0
145	Indietronica	9.0
145	New Wave	5.0
145	Pop	7.0
145	Power Pop	7.0
145	Southern Hip Hop	7.0
145	Synthpop	8.0
145	Trip Hop	7.0
146	Alternative Rock	7.0
146	Art Rock	8.0
146	Britpop	5.0
146	Bubblegum Bass	8.0
146	Conscious Hip Hop	9.0
146	Deconstructed Club	8.0
146	Dream Pop	8.0
146	Electropop	6.0
146	Experimental Hip Hop	9.0
146	Indie Rock	8.0
146	New Wave	5.0
146	Pop	6.0
146	Power Pop	7.0
146	Progressive Rock	7.0
146	Symphonic Rock	7.0
146	Synthpop	6.0
146	Trip Hop	6.0
146	West Coast Hip Hop	9.0
147	Alternative Rock	8.0
147	Art Rock	9.0
147	Bubblegum Bass	6.0
147	Conscious Hip Hop	10.0
147	Deconstructed Club	6.0
147	Electropop	6.0
147	Experimental Hip Hop	6.0
147	Glam Rock	9.0
147	Pop	7.0
147	Pop Rock	9.0
147	Power Pop	6.0
147	Synthpop	6.0
147	Trip Hop	7.0
147	UK Hip Hop	6.0
147	West Coast Hip Hop	10.0
148	Alternative Roc

251	Electronic	9.0
253	Alternative R&B	6.0
253	Art Pop	9.0
253	Conscious Hip Hop	9.0
253	Electronic	9.0
253	Experimental Hip Hop	8.0
253	Jazz Rap	9.0
253	Neo-Soul	6.0
253	UK Hip Hop	8.0
253	West Coast Hip Hop	9.0
257	Alternative R&B	7.0
257	Art Pop	6.0
257	Baroque Pop	5.0
257	Conscious Hip Hop	10.0
257	Electronic	6.0
257	Emo	4.0
257	Experimental Hip Hop	3.0
257	Experimental Rock	5.0
257	Indie Rock	4.0
257	Jazz Rap	10.0
257	Neo-Soul	7.0
257	Post-Punk	5.0
257	Post-Rock	5.0
257	Shoegaze	4.0
257	Singer-Songwriter	7.0
257	Soft Rock	7.0
257	UK Hip Hop	3.0
257	West Coast Hip Hop	10.0
258	Indie Rock	8.0
258	Post-Punk Revival	8.0
260	Art Pop	8.0
260	Conscious Hip Hop	7.0
260	Electronic	8.0
260	Jazz Rap	7.0
260	West Coast Hip Hop	7.0
262	Conscious Hip Hop	10.0
262	Jazz Rap	10.0
262	West Coast Hip Hop	10.0
263	Alternative R&B	9.0
263	Baroque Pop	6.0
263	Conscious Hip Hop	9.0
263	Jazz Rap	9.0
263	Neo-Soul	9.0
263	West Coast Hip Hop	9.0
264	Experimental Rock	6.0
264	Post-Punk	6.0
264	Post-Rock	6.0


In [49]:
X = ratings_df.drop(columns=['Rating'])
y = ratings_df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train['Rating'] = y_train

merged_df = pd.merge(discussions_df, X_train, on='DiscussionID')
merged_df['Subgenre'] = merged_df['Subgenres'].str.split(', ')
merged_df = merged_df.explode('Subgenre')
average_ratings_by_subgenre = merged_df.groupby(['MemberID', 'Subgenre'])['Rating'].mean().reset_index()

y_pred = []
for idx, row in X_test.iterrows():
    subgenres = discussions_df[discussions_df['DiscussionID'] == row['DiscussionID']]['Subgenres'].iloc[0]
    smaller_df = merged_df[merged_df['MemberID'] == row['MemberID']]
    avg_rating = 0
    count = 0
    for sg in subgenres:
        discussions_with_sg = smaller_df[smaller_df['Subgenres'].apply(lambda x: sg in x)]
        if not discussions_with_sg.empty:
            avg_rating += discussions_with_sg['Rating'].mean()
            count += 1
    if count > 0:
        avg_rating /= count
    else:
        avg_rating = X_train['Rating'].mean()  # Use average rating if no discussions attended by the user have the subgenre
    y_pred.append(int(np.round(avg_rating, 0)))

y_pred = np.array(y_pred)
rmse = np.sqrt(np.mean((y_pred - y_test)**2))
print("RMSE:", rmse)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


RMSE: 1.6299193747365521


# Idea 2: Decades

If you think about it, many people will gravitate towards a specific 'era' of music (e.g. I tend to gravitate more towards music from the 2010s and 2020s). Let's see what we can get!

In [50]:
discussions_df['Decade'] = discussions_df['ReleaseYear'].astype(str).str[:3] + '0s'

# Split data into train and test sets
X = ratings_df.drop(columns=['Rating'])
y = ratings_df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train['Rating'] = y_train

# Merge discussions_df with train data
merged_df = pd.merge(discussions_df, X_train, on='DiscussionID')

y_pred = []
for idx, row in X_test.iterrows():
    # Get the decade of the current album
    album_decade = discussions_df.loc[discussions_df['DiscussionID'] == row['DiscussionID'], 'Decade'].iloc[0]
    
    # Filter merged_df to only include discussions attended by the user and in the same decade
    smaller_df = merged_df[(merged_df['MemberID'] == row['MemberID']) & (merged_df['Decade'] == album_decade)]
    
    # Calculate the average rating for discussions in the same decade
    avg_rating = smaller_df['Rating'].mean() if not smaller_df.empty else X_train['Rating'].mean()
    y_pred.append(int(np.round(avg_rating, 0)))

# Calculate RMSE
y_pred = np.array(y_pred)
rmse = np.sqrt(np.mean((y_pred - y_test)**2))
print("RMSE:", rmse)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


RMSE: 1.6833384444561559


# Your Turn: Implement your own content-based filtering!

You can use any of the data already present in the database, or you can use external resources (Musicbrainz, RateYourMusic, Musicboard, Spotify, etc.)

In [ ]:
# TODO: what do you have in mind?